In [1]:
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from scipy.stats.stats import pearsonr   

In [2]:
# Load Data Taxi Fare Data
file_name = 'NYC_Taxi_2013_One_Percent_Fare.csv'
taxi_fare = pd.read_csv(file_name)

In [3]:
# Load Taxi Trip Data 
file_name2 = 'NYC_Taxi_2013_One_Percent_Trip.csv'
taxi_trip = pd.read_csv(file_name2)

In [4]:
# Edit Raw Taxi Fare Dataframe
taxi_fare['tip_percentage'] = taxi_fare['tip_amount'] / taxi_fare['total_amount'] * 100
taxi_fare_df = taxi_fare.drop(['medallion','vendor_id', 'fare_amount','surcharge','mta_tax','tolls_amount'], axis = 1)

taxi_fare_df.head()

,hack_license,pickup_datetime,payment_type,total_amount,tip_amount,tip_percentage
0,A9AE329EA1138052DAC8FDFD8BA86603,3/8/2013 9:55:27 PM,CSH,11.5,0.0,0.000000
1,A9AE329EA1138052DAC8FDFD8BA86603,3/9/2013 6:51:52 PM,CRD,12.0,2.0,16.666667
2,A9AE329EA1138052DAC8FDFD8BA86603,3/9/2013 7:04:35 PM,CSH,18.5,0.0,0.000000
3,A9AE329EA1138052DAC8FDFD8BA86603,3/9/2013 8:33:06 PM,CSH,12.5,0.0,0.000000
4,A9AE329EA1138052DAC8FDFD8BA86603,3/9/2013 11:10:38 PM,CRD,8.4,1.4,16.666667


In [5]:
# Edit Raw Taxi Trip Dataframe
taxi_trip_df = taxi_trip.drop(['medallion', 'vendor_id','rate_code','store_and_fwd_flag','dropoff_datetime'], axis = 1)
taxi_trip_df.head()

,hack_license,pickup_datetime,passenger_count,trip_time_in_secs,trip_distance,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude
0,A9AE329EA1138052DAC8FDFD8BA86603,3/8/2013 9:55:27 PM,2,631,2.5,-73.959610,40.715942,-73.963417,40.692169
1,A9AE329EA1138052DAC8FDFD8BA86603,3/9/2013 6:51:52 PM,2,666,1.8,-73.973373,40.763958,-73.977303,40.782139
2,A9AE329EA1138052DAC8FDFD8BA86603,3/9/2013 7:04:35 PM,2,1401,4.4,-73.977837,40.782818,-74.000893,40.737076
3,A9AE329EA1138052DAC8FDFD8BA86603,3/9/2013 8:33:06 PM,1,751,2.8,-73.980331,40.784382,-73.984291,40.748688
4,A9AE329EA1138052DAC8FDFD8BA86603,3/9/2013 11:10:38 PM,1,261,1.5,-73.986687,40.749474,-73.990677,40.762524


In [6]:
# Merge Two Dataframes
taxi_df = pd.merge(taxi_fare_df, taxi_trip_df, on=['hack_license','pickup_datetime'])
taxi_df.head()

,hack_license,pickup_datetime,payment_type,total_amount,tip_amount,tip_percentage,passenger_count,trip_time_in_secs,trip_distance,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude
0,A9AE329EA1138052DAC8FDFD8BA86603,3/8/2013 9:55:27 PM,CSH,11.5,0.0,0.000000,2,631,2.5,-73.959610,40.715942,-73.963417,40.692169
1,A9AE329EA1138052DAC8FDFD8BA86603,3/9/2013 6:51:52 PM,CRD,12.0,2.0,16.666667,2,666,1.8,-73.973373,40.763958,-73.977303,40.782139
2,A9AE329EA1138052DAC8FDFD8BA86603,3/9/2013 7:04:35 PM,CSH,18.5,0.0,0.000000,2,1401,4.4,-73.977837,40.782818,-74.000893,40.737076
3,A9AE329EA1138052DAC8FDFD8BA86603,3/9/2013 8:33:06 PM,CSH,12.5,0.0,0.000000,1,751,2.8,-73.980331,40.784382,-73.984291,40.748688
4,A9AE329EA1138052DAC8FDFD8BA86603,3/9/2013 11:10:38 PM,CRD,8.4,1.4,16.666667,1,261,1.5,-73.986687,40.749474,-73.990677,40.762524


In [7]:
# Check for NaN
taxi_df.isnull().any()

hack_license         False
pickup_datetime      False
payment_type         False
total_amount         False
tip_amount           False
tip_percentage        True
passenger_count      False
trip_time_in_secs    False
trip_distance        False
pickup_longitude     False
pickup_latitude      False
dropoff_longitude    False
dropoff_latitude     False
dtype: bool

In [8]:
# Fill in NaN
taxi_df = taxi_df.fillna(0)

In [9]:
taxi_df.isnull().any()

hack_license         False
pickup_datetime      False
payment_type         False
total_amount         False
tip_amount           False
tip_percentage       False
passenger_count      False
trip_time_in_secs    False
trip_distance        False
pickup_longitude     False
pickup_latitude      False
dropoff_longitude    False
dropoff_latitude     False
dtype: bool

In [10]:
# Discretize Tip Percentage
tip_per = taxi_df['tip_percentage']
tipp_discretize = pd.cut(tip_per, bins=[0, 15, 20, 100], labels = ['Low', 'Mid', 'High'], include_lowest = True)

taxi_df['tip_rate'] = tipp_discretize
taxi_df.head()

,hack_license,pickup_datetime,payment_type,total_amount,tip_amount,tip_percentage,passenger_count,trip_time_in_secs,trip_distance,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,tip_rate
0,A9AE329EA1138052DAC8FDFD8BA86603,3/8/2013 9:55:27 PM,CSH,11.5,0.0,0.000000,2,631,2.5,-73.959610,40.715942,-73.963417,40.692169,Low
1,A9AE329EA1138052DAC8FDFD8BA86603,3/9/2013 6:51:52 PM,CRD,12.0,2.0,16.666667,2,666,1.8,-73.973373,40.763958,-73.977303,40.782139,Mid
2,A9AE329EA1138052DAC8FDFD8BA86603,3/9/2013 7:04:35 PM,CSH,18.5,0.0,0.000000,2,1401,4.4,-73.977837,40.782818,-74.000893,40.737076,Low
3,A9AE329EA1138052DAC8FDFD8BA86603,3/9/2013 8:33:06 PM,CSH,12.5,0.0,0.000000,1,751,2.8,-73.980331,40.784382,-73.984291,40.748688,Low
4,A9AE329EA1138052DAC8FDFD8BA86603,3/9/2013 11:10:38 PM,CRD,8.4,1.4,16.666667,1,261,1.5,-73.986687,40.749474,-73.990677,40.762524,Mid


In [11]:
target = taxi_df['tip_rate']
payment_type = taxi_df['payment_type']
pd.crosstab(target, payment_type)

payment_type,CRD,CSH,DIS,NOC,UNK
tip_rate,,,,,
Low,299791,778194,1165,4024,851
Mid,559127,42,7,4,1003
High,59781,11,1,1,63


In [12]:
# Converting Datetime Pickup to Datetime object
pudt = taxi_df['pickup_datetime']

In [13]:
# dt_obj = datetime.strptime(pudt, '%m/%d/%Y %I:%M:%S %p')

dt_convt = []

for dt in pudt: 
    dt_obj = datetime.strptime(dt, '%m/%d/%Y %I:%M:%S %p')
    dt_convt.append(dt_obj)
    

In [14]:
taxi_df['pickup_datetime'] = dt_convt

In [15]:
taxi_df.head()

,hack_license,pickup_datetime,payment_type,total_amount,tip_amount,tip_percentage,passenger_count,trip_time_in_secs,trip_distance,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,tip_rate
0,A9AE329EA1138052DAC8FDFD8BA86603,2013-03-08 21:55:27,CSH,11.5,0.0,0.000000,2,631,2.5,-73.959610,40.715942,-73.963417,40.692169,Low
1,A9AE329EA1138052DAC8FDFD8BA86603,2013-03-09 18:51:52,CRD,12.0,2.0,16.666667,2,666,1.8,-73.973373,40.763958,-73.977303,40.782139,Mid
2,A9AE329EA1138052DAC8FDFD8BA86603,2013-03-09 19:04:35,CSH,18.5,0.0,0.000000,2,1401,4.4,-73.977837,40.782818,-74.000893,40.737076,Low
3,A9AE329EA1138052DAC8FDFD8BA86603,2013-03-09 20:33:06,CSH,12.5,0.0,0.000000,1,751,2.8,-73.980331,40.784382,-73.984291,40.748688,Low
4,A9AE329EA1138052DAC8FDFD8BA86603,2013-03-09 23:10:38,CRD,8.4,1.4,16.666667,1,261,1.5,-73.986687,40.749474,-73.990677,40.762524,Mid


In [16]:
taxi_df.to_csv('Taxi_df.csv')